In [2]:
import sys
sys.path.append("..")

In [3]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.3-70b-versatile",
    temperature=0.6
)

response = llm.invoke("Explain RAG in one sentence.")
print(response.content)

c:\Users\Admin\Desktop\Coding\RAG System\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RAG (Retrieve, Augment, Generate) is an artificial intelligence framework that combines retrieval and generation techniques to produce more accurate and informative responses by retrieving relevant information from a knowledge base and then using that information to generate a response.


In [4]:
from langchain_community.utilities import SQLDatabase

from sql_alchemy.engine import engine
 
db = SQLDatabase(engine)

Connection successful!
